# Create the main life-cycle inventory database in your brightway project

In [1]:
import bw2data as bd
from premise import *
from mescal import *
from carculator_truck import *
from datetime import datetime

## Import premise databases

In [2]:
ei_version = '3.10.1' # version of the ecoinvent database. Careful: carculator_truck v0.5.0 is compatible with ecoinvent 3.10 only
ei_db_name = f"ecoinvent-{ei_version}-cutoff" # name of the ecoinvent database in the brightway project
bd.projects.set_current(f'ecoinvent{ei_version}') # set the current brightway project
model = 'image' # IAM used to create the database
pathway = 'SSP2-Base' # SSP-RCP scenario used to create the database
years = [2020, 2050] # years of the database

In [3]:
if ei_version == '3.10.1':
    ei_version_premise = '3.10'
else:
    ei_version_premise = ei_version 

In [4]:
# Clear cache is encouraged if updating premise or if encountering issues with inventories
clear_cache()

Cache folder cleared!


In [5]:
# Initialize the premise database
ndb = NewDatabase(
    scenarios=[{"model": model, "pathway": pathway, "year": year} for year in years],
    source_db=ei_db_name,
    source_version=ei_version_premise,
    key='xxx', # ask a key to Romain Sacchi (romain.sacchi@psi.ch)
    biosphere_name='biosphere3',
)

premise v.(2, 2, 6)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

100%|██████████| 23523/23523 [00:00<00:00, 81626.31it/s]


Adding exchange data to activities


100%|██████████| 743409/743409 [01:17<00:00, 9560.19it/s] 


Filling out exchange data


100%|██████████| 23523/23523 [00:05<00:00, 4207.00it/s] 


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Correct missing flow categories for biosphere exchanges
Remove empty exchanges.
Remove uncertainty data.
- Extracting inventories
Cannot find cached inventories. Will create them now for next time...
Importing default inventories...

Extracted 1 worksheets in 0.61 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Remove uncertainty data.
Extracted 1 worksheets in 0.07 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Remove unce

In [6]:
# Update the database with IAM data
ndb.update()

Processing scenarios for all sectors: 100%|█| 2/2 [14:15<00:00, 427.74

Done!



In [7]:
# Write the database in your brightway project
ndb.write_db_to_brightway(name=[f'ecoinvent_cutoff_{ei_version}_{model}_{pathway}_{year}' for year in years])

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:54


Title: Writing activities to SQLite3 database:
  Started: 01/31/2025 16:46:05
  Finished: 01/31/2025 16:46:59
  Total time elapsed: 00:00:54
  CPU %: 91.30
  Memory %: 33.97
Created database: ecoinvent_cutoff_3.10.1_image_SSP2-Base_2020
Running all checks...
Minor anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:39


Title: Writing activities to SQLite3 database:
  Started: 01/31/2025 16:54:19
  Finished: 01/31/2025 16:55:59
  Total time elapsed: 00:01:39
  CPU %: 89.10
  Memory %: 31.85
Created database: ecoinvent_cutoff_3.10.1_image_SSP2-Base_2050
Generate scenario report.
Report saved under C:\Users\matth\PycharmProjects\energyscope-lca\01_Notebooks\export\scenario_report.
Generate change report.
Report saved under C:\Users\matth\PycharmProjects\energyscope-lca\01_Notebooks.


## Minor corrections to premise databases using mescal 

In [8]:
for year in years:
    remove_quebec_flow_in_global_heat_market(
        db_name=f'ecoinvent_cutoff_{ei_version}_{model}_{pathway}_{year}',
        activity_name='market group for heat, district or industrial, other than natural gas'
    ) # Remove the Quebec heat flow in the global heat market activity

In [9]:
for year in years:
    activity_name = 'farming and supply of corn'
    activities_to_change = [i for i in bd.Database(f'ecoinvent_cutoff_{ei_version}_{model}_{pathway}_{year}').search(activity_name, limit=1000) if i['name'] == activity_name] # for all locations
    flow_to_change = [i for i in bd.Database('biosphere3').search('Water, unspecified natural origin') if 
                      (i['name'] == 'Water, unspecified natural origin')
                      & (i['categories'] == ('natural resource', 'in water'))
                      ][0]
    flow_to_change_code = flow_to_change['code']
    for act in activities_to_change:
        change_flow_value(
            activity_code=act['code'],
            flow_code=flow_to_change_code,
            database_name=f'ecoinvent_cutoff_{ei_version}_{model}_{pathway}_{year}',
            new_value=1.222,
            flow_type='biosphere',
        )

## Import carculator_truck databases

In [10]:
def create_truck_database(cycle, year):
    tip = TruckInputParameters()
    tip.static()
    dcts, array = fill_xarray_from_input_parameters(tip, scope={"year":[year]})
    tm = TruckModel(array, cycle=cycle)
    tm.set_all()
    ic = InventoryTruck(tm)
    
    i = ic.export_lci(
        software="brightway2",
        ecoinvent_version=ei_version_premise,
        format="bw2io",
        filename=cycle.lower(),
    )
    
    i.apply_strategies()

    i.match_database(fields=["name", "unit", "location"])
    i.match_database(ei_db_name,  fields=["reference product", "name", "unit", "location"])
    i.match_database('biosphere3',  fields=["name", "unit", "categories"])

    i.statistics()
    
    i.drop_unlinked(i_am_reckless=True)  # remove noise elementary flows (not characterized)

    if cycle.lower() + f"_truck_{datetime.now().strftime('%Y%m%d')}_{year}" in bd.databases:
        del bd.databases[cycle.lower() + f"_truck_{datetime.now().strftime('%Y%m%d')}_{year}"]
    
    i.write_database()

In [11]:
for year in years:
    create_truck_database(cycle='Urban delivery', year=year) # 150 km 
    create_truck_database(cycle='Regional delivery', year=year) # 400 km 

Finding solutions for trucks...
Urban delivery driving cycle is selected. 
Vehicles will be designed to achieve a minimal range of 150 km.


'-' vehicle with driving mass superior to the permissible gross weight.
'/' vehicle not available for the specified year.
+-------------------+-----+-----+------+-----+-----+------+------+
| Payload (in tons) | 18t | 26t | 3.5t | 32t | 40t | 60t  | 7.5t |
+-------------------+-----+-----+------+-----+-----+------+------+
|     BEV, 2020     | 2.7 | 6.3 | 0.8  | 8.8 | 8.8 | 16.4 | 1.8  |
|     FCEV, 2020    | 2.7 | 6.3 | 0.8  | 8.8 | 8.8 | 16.4 | 1.8  |
|    HEV-d, 2020    | 2.7 | 6.3 | 0.8  | 8.8 | 8.8 | 16.4 | 1.8  |
|    ICEV-d, 2020   | 2.7 | 6.3 | 0.8  | 8.8 | 8.8 | 16.4 | 1.8  |
|    ICEV-g, 2020   | 2.7 | 6.3 | 0.8  | 8.8 | 8.8 | 16.4 | 1.8  |
|    PHEV-d, 2020   | 2.7 | 6.3 | 0.8  | 8.8 | 8.8 | 16.4 | 1.8  |
+-------------------+-----+-----+------+-----+-----+------+------+
****************** IMPORTANT BACKGROUND PARAMETERS ****************

0% [##############################] 100% | ETA: 00:00:00 | Item ID: 1801
Total time elapsed: 00:00:01


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: assign_only_product_as_production
Applying strategy: strip_biosphere_exc_locations
Applied 4 strategies in 0.01 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
537 datasets
8651 exchanges
1008 unlinked exchanges
  Type biosphere: 24 unique unlinked exchanges
Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/31/2025 17:03:27
  Finished: 01/31/2025 17:03:28
  Total time elapsed: 00:00:00
  CPU %: 93.20
  Memory %: 28.04
Created database: urban delivery_truck_20250131_2020
Finding solutions for trucks...
Regional delivery driving cycle is selected. 
Vehicles will be designed to achieve a minimal range of 400 km.


'-' vehicle with driving mass superior to the permissible gross weight.
'/' vehicle not available for the specified year.
+-------------------+-----+-----+------+------+------+------+------+
| Payload (in tons) | 18t | 26t | 3.5t | 32t  | 40t  | 60t  | 7.5t |
+-------------------+-----+-----+------+------+------+------+------+
|     BEV, 2020     | 3.2 | 6.3 | 0.8  | 10.3 | 10.3 | 19.3 | 1.8  |
|     FCEV, 2020    | 3.2 | 6.3 | 0.8  | 10.3 | 10.3 | 19.3 | 1.8  |
|    HEV-d, 2020    | 3.2 | 6.3 | 0.8  | 10.3 | 10.3 | 19.3 | 1.8  |
|    ICEV-d, 2020   | 3.2 | 6.3 | 0.8  | 10.3 | 10.3 | 19.3 | 1.8  |
|    ICEV-g, 2020   | 3.

...

Title: Writing activities to SQLite3 database:
  Started: 01/31/2025 17:06:34
  Finished: 01/31/2025 17:06:35
  Total time elapsed: 00:00:00
  CPU %: 92.90
  Memory %: 26.87
Created database: urban delivery_truck_20250131_2050
Finding solutions for trucks...
Regional delivery driving cycle is selected. 
Vehicles will be designed to achieve a minimal range of 400 km.


'-' vehicle with driving mass superior to the permissible gross weight.
'/' vehicle not available for the specified year.
+-------------------+-----+-----+------+------+------+------+------+
| Payload (in tons) | 18t | 26t | 3.5t | 32t  | 40t  | 60t  | 7.5t |
+-------------------+-----+-----+------+------+------+------+------+
|     BEV, 2050     | 3.2 | 6.3 | 0.8  | 10.3 | 10.3 | 19.3 | 1.8  |
|     FCEV, 2050    | 3.2 | 6.3 | 0.8  | 10.3 | 10.3 | 19.3 | 1.8  |
|    HEV-d, 2050    | 3.2 | 6.3 | 0.8  | 10.3 | 10.3 | 19.3 | 1.8  |
|    ICEV-d, 2050   | 3.2 | 6.3 | 0.8  | 10.3 | 10.3 | 19.3 | 1.8  |
|    ICEV-g, 2050   | 3.

0% [##############################] 100% | ETA: 00:00:00 | Item ID: 1801
Total time elapsed: 00:00:01


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: assign_only_product_as_production
Applying strategy: strip_biosphere_exc_locations
Applied 4 strategies in 0.01 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
537 datasets
8590 exchanges
1008 unlinked exchanges
  Type biosphere: 24 unique unlinked exchanges
Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/31/2025 17:07:57
  Finished: 01/31/2025 17:07:58
  Total time elapsed: 00:00:00
  CPU %: 98.30
  Memory %: 27.29
Created database: regional delivery_truck_20250131_2050


## Minor changes in naming to the carculator_truck databases

In [12]:
def rename_truck_activities(cycle, year):
    db = bd.Database(f"{cycle.lower()}_truck_{datetime.now().strftime('%Y%m%d')}_{year}")
    db_list = [a for a in db]
    for act in db_list:
        if act.as_dict()['name'].startswith('transport, truck') or act.as_dict()['name'].startswith('truck,'):
            act.as_dict()['name'] += f', {cycle.lower()}'
        act.save()

In [13]:
for year in years:
    rename_truck_activities(cycle='Regional delivery', year=year)
    rename_truck_activities(cycle='Urban delivery', year=year)

## Merge the premise and carculator_truck databases into a single database

In [14]:
for year in years:
    main_db = Database(f'ecoinvent_cutoff_{ei_version}_{model}_{pathway}_{year}')
    carculator_truck_urban_db = Database(f"urban delivery_truck_{datetime.now().strftime('%Y%m%d')}_{year}")
    carculator_truck_regional_db = Database(f"regional delivery_truck_{datetime.now().strftime('%Y%m%d')}_{year}")
    
    db = main_db + carculator_truck_urban_db + carculator_truck_regional_db  # concatenation of the database (the lists of dictionaries are simply added up)
    
    db.merge(
        main_ecoinvent_db_name=f'ecoinvent_cutoff_{ei_version}_{model}_{pathway}_{year}',
        old_main_db_names=[f'ecoinvent-{ei_version}-cutoff'],
        new_db_name=f'ecoinvent_cutoff_{ei_version}_{model}_{pathway}_{year}+truck_carculator',
        write=True,
        check_duplicates=True,
    )  # performs relinking of secondary databases towards the main database, as well as elimination of duplicates 

Getting activity data


100%|██████████| 34821/34821 [00:00<00:00, 80235.85it/s]


Adding exchange data to activities


100%|██████████| 1001263/1001263 [01:56<00:00, 8619.39it/s] 


Filling out exchange data


...

Filling out exchange data


100%|██████████| 537/537 [00:01<00:00, 358.14it/s]


Loaded regional delivery_truck_20250131_2050 from brightway!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:33


Title: Writing activities to SQLite3 database:
  Started: 01/31/2025 17:42:51
  Finished: 01/31/2025 17:45:25
  Total time elapsed: 00:02:33
  CPU %: 82.80
  Memory %: 46.90
ecoinvent_cutoff_3.10.1_image_SSP2-Base_2050+truck_carculator written to Brightway!


## Regionalization of the database

To regionalize the database clone the [Regiopremise](https://github.com/matthieu-str/Regiopremise) repository and run the [demo.ipynb](https://github.com/matthieu-str/Regiopremise/blob/ei3.10/doc/demo.ipynb) notebook. 